# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234942690399                   -0.50    7.0
  2   -7.250214010614       -1.82       -1.40    1.0
  3   -7.251235776830       -2.99       -2.01    4.0
  4   -7.251098626927   +   -3.86       -1.97    4.0
  5   -7.251327390519       -3.64       -2.63    4.0
  6   -7.251337908327       -4.98       -3.11    3.0
  7   -7.251338791186       -6.05       -4.12    3.0
  8   -7.251338795461       -8.37       -4.28    4.0
  9   -7.251338798602       -8.50       -4.89    3.0
 10   -7.251338798662      -10.22       -5.19    1.0
 11   -7.251338798703      -10.39       -5.90    3.0
 12   -7.251338798704      -11.85       -6.33    2.0
 13   -7.251338798705      -12.68       -6.82    4.0
 14   -7.251338798705      -13.65       -7.33    2.0
 15   -7.251338798705      -15.05       -7.73    2.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.07248379398601269
[ Info: Arnoldi iteration step 2: normres = 0.4152047750152431
[ Info: Arnoldi iteration step 3: normres = 0.9539720357542801
[ Info: Arnoldi iteration step 4: normres = 0.26896664513073965
[ Info: Arnoldi iteration step 5: normres = 0.6353109603879212
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (4.97e-02, 8.68e-02, 5.86e-01, 2.24e-01, 1.65e-02)
[ Info: Arnoldi iteration step 6: normres = 0.21967851110237588
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (8.59e-03, 1.28e-01, 1.10e-01, 1.11e-01, 7.94e-02)
[ Info: Arnoldi iteration step 7: normres = 0.07172060502561016
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (2.91e-04, 8.18e-03, 5.74e-03, 3.84e-02, 4.95e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11253190267360703
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.39e